In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from utils import mnist

In [2]:
train_loader, test_loader = mnist()

In [3]:
class Net(nn.Module):
    def __init__(self, log_softmax=False):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)
        self.log_softmax = log_softmax
        self.optim = optim.Adam(self.parameters(), lr=0.01)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        if self.log_softmax:
            x = F.log_softmax(x, dim=1)
        else:
            x = torch.log(F.softmax(x, dim=1))
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = F.nll_loss(output, target, **kwargs)
        return self._loss

In [4]:
def train(epoch, models):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        for model in models:
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader))
            losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader))
        losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
        print(line + losses)

In [5]:
models = [Net(), Net(True)]

In [6]:
avg_lambda = lambda l: 'Loss: {:.4f}'.format(l)
acc_lambda = lambda c, p: 'Accuracy: {}/{} ({:.0f}%)'.format(c, len(test_loader.dataset), p)
line = lambda i, l, c, p: '{}: '.format(i) + avg_lambda(l) + '\t' + acc_lambda(c, p)

def test(models):
    test_loss = [0]*len(models)
    correct = [0]*len(models)
    with torch.no_grad():
        for data, target in test_loader:
            output = [m(data) for m in models]
            for i, m in enumerate(models):
                test_loss[i] += m.loss(output[i], target, size_average=False).item() # sum up batch loss
                pred = output[i].data.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct[i] += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    for i in range(len(models)):
        test_loss[i] /= len(test_loader.dataset)
    correct_pct = [100. * c / len(test_loader.dataset) for c in correct]
    lines = '\n'.join([line(i, test_loss[i], correct[i], correct_pct[i]) for i in range(len(models))]) + '\n'
    report = 'Test set:\n' + lines
    
    print(report)

In [7]:
for epoch in range(1, 21):
    train(epoch, models)
    test(models)

Train Epoch: 1 [0/60000 (0%)]	Losses 0: 2.339125 1: 2.295040
Train Epoch: 1 [10000/60000 (17%)]	Losses 0: 0.187201 1: 0.092009
Train Epoch: 1 [20000/60000 (33%)]	Losses 0: 0.221736 1: 0.339246
Train Epoch: 1 [30000/60000 (50%)]	Losses 0: 0.394539 1: 0.436334
Train Epoch: 1 [40000/60000 (67%)]	Losses 0: 0.382679 1: 0.426834
Train Epoch: 1 [50000/60000 (83%)]	Losses 0: 0.091231 1: 0.119856
Train Epoch: 1 [60000/60000 (100%)]	Losses 0: 0.096878 1: 0.122670
Test set:
0: Loss: 0.2347	Accuracy: 9257/10000 (92%)
1: Loss: 0.2112	Accuracy: 9354/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Losses 0: 0.183473 1: 0.141665
Train Epoch: 2 [10000/60000 (17%)]	Losses 0: 0.160425 1: 0.130580
Train Epoch: 2 [20000/60000 (33%)]	Losses 0: 0.163106 1: 0.238637
Train Epoch: 2 [30000/60000 (50%)]	Losses 0: 0.185041 1: 0.147429
Train Epoch: 2 [40000/60000 (67%)]	Losses 0: 0.349799 1: 0.308022
Train Epoch: 2 [50000/60000 (83%)]	Losses 0: 0.218220 1: 0.233128
Train Epoch: 2 [60000/60000 (100%)]	Losses 0: 0.150790

Train Epoch: 15 [60000/60000 (100%)]	Losses 0: 0.076751 1: 0.199879
Test set:
0: Loss: 0.1462	Accuracy: 9575/10000 (95%)
1: Loss: 0.1423	Accuracy: 9572/10000 (95%)

Train Epoch: 16 [0/60000 (0%)]	Losses 0: 0.167547 1: 0.172187
Train Epoch: 16 [10000/60000 (17%)]	Losses 0: 0.021523 1: 0.186436
Train Epoch: 16 [20000/60000 (33%)]	Losses 0: 0.041921 1: 0.114260
Train Epoch: 16 [30000/60000 (50%)]	Losses 0: 0.139912 1: 0.384291
Train Epoch: 16 [40000/60000 (67%)]	Losses 0: 0.050683 1: 0.109094
Train Epoch: 16 [50000/60000 (83%)]	Losses 0: 0.195850 1: 0.096648
Train Epoch: 16 [60000/60000 (100%)]	Losses 0: 0.181701 1: 0.211595
Test set:
0: Loss: 0.1505	Accuracy: 9553/10000 (95%)
1: Loss: 0.1431	Accuracy: 9584/10000 (95%)

Train Epoch: 17 [0/60000 (0%)]	Losses 0: 0.024163 1: 0.014596
Train Epoch: 17 [10000/60000 (17%)]	Losses 0: 0.047598 1: 0.019603
Train Epoch: 17 [20000/60000 (33%)]	Losses 0: 0.119206 1: 0.127749
Train Epoch: 17 [30000/60000 (50%)]	Losses 0: 0.302964 1: 0.113433
Train Epoc